In [28]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image
from pandas import get_dummies
from sklearn.model_selection import train_test_split
# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

In [29]:
df = pd.read_csv('coffeeBotData-Cleaned.csv', index_col=0)
df.head()

,clouds,humidity,temp,type,visibility,windSpeed,weatherCond_Clear,weatherCond_Clouds,weatherCond_Fog,weatherCond_Haze,weatherCond_Rain,weatherCond_Smoke,weatherCond_Snow,weatherCond_Thunderstorm
FFB69BDF-D8CC-4E12-A944-3AC7FB582383,1,45,81.0,0,10.0,3.0,1,0,0,0,0,0,0,0
B21714A4-BD63-4BA2-9296-65C466681513,1,83,70.0,0,10.0,2.0,1,0,0,0,0,0,0,0
1E8A0181-D13B-469B-AA49-DA16EFF6C227,1,33,73.0,0,10.0,9.0,1,0,0,0,0,0,0,0
C1E16B18-8396-4031-A312-22C70D405C6C,75,80,39.0,0,10.0,12.0,0,1,0,0,0,0,0,0
C0784390-B4F3-49A4-8E99-2CB8A4D6FE51,90,82,70.0,0,10.0,9.0,0,1,0,0,0,0,0,0


In [30]:
#move type to back
df1 = df.pop('type')
df['type']=df1

cols = df.columns
features = cols[0:-1]
labels = cols[-1]
print(features)
print(labels)

Index(['clouds', 'humidity', 'temp', 'visibility', 'windSpeed',
       'weatherCond_Clear', 'weatherCond_Clouds', 'weatherCond_Fog',
       'weatherCond_Haze', 'weatherCond_Rain', 'weatherCond_Smoke',
       'weatherCond_Snow', 'weatherCond_Thunderstorm'],
      dtype='object')
type


In [31]:
#Well conditioned data will have zero mean and equal variance
#We get this automattically when we calculate the Z Scores for the data

data_norm = pd.DataFrame(df)

for feature in features:
    df[feature] = (df[feature] - df[feature].mean())/df[feature].std()

#Show that should now have zero mean
print("Averages")
print(df.mean())

print("\n Deviations")
#Show that we have equal variance
print(pow(df.std(),2))

Averages
clouds                      7.612429e-16
humidity                   -2.893981e-16
temp                       -2.026157e-16
visibility                 -1.404432e-16
windSpeed                  -2.331468e-16
weatherCond_Clear          -2.509104e-16
weatherCond_Clouds         -5.662137e-17
weatherCond_Fog             6.232052e-16
weatherCond_Haze            2.235249e-16
weatherCond_Rain            2.335169e-16
weatherCond_Smoke           4.880355e-18
weatherCond_Snow            3.892719e-16
weatherCond_Thunderstorm    1.113924e-16
type                        5.000000e-01
dtype: float64

 Deviations
clouds                      1.000000
humidity                    1.000000
temp                        1.000000
visibility                  1.000000
windSpeed                   1.000000
weatherCond_Clear           1.000000
weatherCond_Clouds          1.000000
weatherCond_Fog             1.000000
weatherCond_Haze            1.000000
weatherCond_Rain            1.000000
weatherCond_Smoke  

In [32]:
#Shuffle The data
indices = data_norm.index.tolist()
indices = np.array(indices)
np.random.shuffle(indices)
X = data_norm.reindex(indices)[features]
y = data_norm.reindex(indices)[labels]

In [33]:
# One Hot Encode as a dataframe
y = get_dummies(y)

# Generate Training and Validation Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Convert to np arrays so that we can use with TensorFlow
X_train = np.array(X_train).astype(np.float32)
X_test  = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test  = np.array(y_test).astype(np.float32)

In [34]:
#Check to make sure split still has 4 features and 3 labels
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(402, 13) (402, 2)
(198, 13) (198, 2)


In [62]:
training_size = X_train.shape[1]
test_size = X_test.shape[1]
num_features = 13
num_labels = 2


num_hidden = 10
graph = tf.Graph()
with graph.as_default():
    tf_train_set    = tf.constant(X_train)
    tf_train_labels = tf.constant(y_train)
    tf_valid_set    = tf.constant(X_test)
 
    
    print(tf_train_set)
    print(tf_train_labels)
    
    ## Note, since there is only 1 layer there are actually no hidden layers... but if there were
    ## there would be num_hidden
    weights_1 = tf.Variable(tf.truncated_normal([num_features, num_hidden], name="weights_1"))
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], name="weights_2"))
    ## tf.zeros Automaticaly adjusts rows to input data batch size
    bias_1 = tf.Variable(tf.zeros([num_hidden], name="bias_1"))
    bias_2 = tf.Variable(tf.zeros([num_labels], name="bias_2"))
    saver = tf.train.Saver()


    
    logits_1 = tf.matmul(tf_train_set , weights_1, name="logits_1" ) + bias_1
    rel_1 = tf.nn.relu(logits_1, name="rel_1")
    logits_2 = tf.matmul(rel_1, weights_2, name="logits_2") + bias_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(.1).minimize(loss)
    
    
    ## Training prediction
    predict_train = tf.nn.softmax(logits_2)
    
    
    
    # Validation prediction
    logits_1_val = tf.matmul(tf_valid_set, weights_1) + bias_1
    rel_1_val    = tf.nn.relu(logits_1_val)
    logits_2_val = tf.matmul(rel_1_val, weights_2) + bias_2
    predict_valid = tf.nn.softmax(logits_2_val)

Tensor("Const:0", shape=(402, 13), dtype=float32)
Tensor("Const_1:0", shape=(402, 2), dtype=float32)


In [63]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 10000
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print(loss.eval())
    for step in range(num_steps):
        _,l, predictions = session.run([optimizer, loss, predict_train])
        
        if (step % 2000 == 0):
            #print(predictions[3:6])
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy( predictions, y_train[:, :]))
            print('Validation accuracy: %.1f%%' % accuracy(predict_valid.eval(), y_test))
            save_path = saver.save(session, "/tmp/model.ckpt")
            print("Model saved in path: %s" % save_path)

2.278749
Loss at step 0: 2.278749
Training accuracy: 51.2%
Validation accuracy: 51.5%
Model saved in path: /tmp/model.ckpt
Loss at step 2000: 0.420242
Training accuracy: 80.6%
Validation accuracy: 72.7%
Model saved in path: /tmp/model.ckpt
Loss at step 4000: 0.393381
Training accuracy: 82.6%
Validation accuracy: 71.7%
Model saved in path: /tmp/model.ckpt
Loss at step 6000: 0.370464
Training accuracy: 84.6%
Validation accuracy: 71.7%
Model saved in path: /tmp/model.ckpt
Loss at step 8000: 0.358558
Training accuracy: 83.3%
Validation accuracy: 73.2%
Model saved in path: /tmp/model.ckpt


In [64]:
 tf.saved_model.simple_save(session,
    "",inputs={
        "weights_1": weights_1,
        "weights_2": weights_2,
        "bias_1":bias_1,
        "bias_2":bias_2
    },outputs={
        "logits_1": logits_1,
         "rel_1":rel_1,
         "logits_2":logits_2})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model.pb


In [41]:
checkpoint = tf.train.get_checkpoint_state("/tmp/")
input_checkpoint = checkpoint.model_checkpoint_path

In [42]:
!pip install tensorflowjs

    100% |████████████████████████████████| 24.4MB 52kB/s eta 0:00:011
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 307kB 3.2MB/s eta 0:00:01
    100% |████████████████████████████████| 6.1MB 245kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 5.6MB/s ta 0:00:01
    100% |████████████████████████████████| 59.4MB 22kB/s eta 0:00:011
    100% |████████████████████████████████| 51kB 8.4MB/s eta 0:00:01
    100% |████████████████████████████████| 3.0MB 448kB/s ta 0:00:011
  Found existing installation: numpy 1.14.2
    Uninstalling numpy-1.14.2:
      Successfully uninstalled numpy-1.14.2
  Found existing installation: six 1.10.0
    DEPRECATION: Uninstalling a distutils installed project (six) has been deprecated and will be removed in a future version. This is due to th

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='model' \
    --saved_model_tags=serve \
    /saved_model.pb \
    /"my_model"

In [47]:
tf.saved_model.simple_save(session, export_dir="/", )


TypeError: simple_save() missing 3 required positional arguments: 'export_dir', 'inputs', and 'outputs'